In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score,accuracy_score,recall_score,f1_score, confusion_matrix

In [2]:
#BUSCAR EN PARALELO @
# VP: 2 (1,1); FN: 1 (1,0); FP: 0 (0,1); VN: 2 (0,0)
# vector_prueba = [1,2,3,4,5,6,4,3,2,1,2,3,4,4,10,11,19,25,39,40,2,3,4,10,15,35]
# pronos_prueba = [1,0,0,0,0,0,0,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,1,0,0,1]

#MIRAR MAXIMOS LOCALES (CONCAVIDAD)
#TRANSFORMACIONES
#PRONOSTICO

# TAREAS
#SUBIR A POSTGRES

In [3]:
#TRAERME VARIABLES
#TRAER 
#NOMINALES EN MONEDA LOCAL
#VER PERCENTILES Y DOBLES MAXIMOS

## Modelo de Kaminsky para la predicción de crisis financieras

### 1. Funciones

Este código implementa el modelo de Kaminsky para la predicción de crisis financieras, utilizando diferentes métricas de evaluación, como la precisión, exactitud, sensibilidad, F1 y el score de Kaminsky (SP). A continuación, se describe cada una de las funciones utilizadas en el código:

1. **sp_score:** Esta función calcula el score de Kaminsky (SP) dado un conjunto de valores reales y predichos. El score SP mide la calidad de las predicciones al compararlas con los resultados reales.

2. **scores:** Esta función calcula varias métricas de evaluación, como la precisión, exactitud, sensibilidad, F1 y el score SP, dadas las predicciones y los resultados reales.

3. **percentiles:** Esta función devuelve una lista de tuplas (percentil, valor) para un conjunto de datos dado. Los percentiles se calculan utilizando pasos especificados por el parámetro 'steps'.

4. **prediction:** Esta función genera predicciones basadas en los umbrales (percentiles) proporcionados. Si el valor de un indicador económico es mayor/menor que el umbral especificado, se predice una crisis financiera, y viceversa.

5. **list_predictions:** Esta función crea un diccionario que contiene las predicciones para cada par (percentil, valor) en el conjunto de datos.

6. **kaminsky_metricas:** Esta función calcula varias métricas de evaluación para cada par (percentil, valor) en el conjunto de datos y devuelve un DataFrame con los resultados.

7. **kaminsky_max:** Esta función busca el umbral que maximiza el score SP (o cualquier otra métrica especificada) en el conjunto de datos. Además, permite elegir la forma en que se selecciona el umbral máximo: primer umbral, último umbral o umbral intermedio.

En resumen, este código implementa un modelo de Kaminsky para predecir crisis financieras utilizando indicadores económicos y busca el umbral óptimo para maximizar la calidad de las predicciones. Cada función en el código tiene un propósito específico y se utilizan en conjunto para lograr el objetivo final de encontrar el mejor umbral y evaluar la calidad de las predicciones.

In [4]:
#################################################################################################################
# Función sp_score: Calcula el score de Kaminsky (SP) para las predicciones
def sp_score(ytest, ypred):
    # Calcular la matriz de confusión entre los valores reales y las predicciones
    M = confusion_matrix(ytest, ypred)

    # Calcular el denominador de las tasas de error E1 y E2
    DEN_E1 = (M[0,1]+M[0,0])
    DEN_E2 = (M[1,0]+M[1,1])

    E1 = 0
    E2 = 0

    # Calcular la tasa de error E1, que es la proporción de falsos positivos
    if DEN_E1 == 0:
        E1 = 0
    else:
        E1 = M[0,1]/DEN_E1

    # Calcular la tasa de error E2, que es la proporción de falsos negativos
    if DEN_E2 == 0:
        E2 = 0
    else:
        E2 = M[1,0]/DEN_E2    

    # Calcular el score de Kaminsky (SP) a partir de las tasas de error E1 y E2
    SP = 1-(E1+E2)
    return SP

# Función scores: Calcula diversas métricas de evaluación para las predicciones
def scores(ytest, ypred):
    # Calcular las métricas de evaluación y almacenarlas en una lista
    listas  = [precision_score(ytest, ypred), 
                accuracy_score(ytest, ypred), 
                recall_score(ytest, ypred),
                f1_score(ytest, ypred),
                sp_score(ytest, ypred)]

    # Crear un DataFrame con las métricas de evaluación y sus respectivos nombres
    A = pd.DataFrame(listas, index = ["precision_score", "accuracy_score", "recall_score", "F1_score", "SP_score"])
    return A

#################################################################################################################
# Función percentiles: Extrae los percentiles de un conjunto de datos
def percentiles(X, steps = 1):
    l = []
    # Calcular los percentiles del conjunto de datos en pasos especificados
    for percentil in range(0, 101, steps):
        value_percentil = np.percentile(X, percentil)
        pareja = (percentil, value_percentil)
        l.append(pareja)
    return l

#################################################################################################################
# Función prediction: Genera predicciones basadas en los umbrales (percentiles) proporcionados
def prediction(X, value_percentil, direction):
    predict = []

    # Si la dirección es "lt" (menor que), predecir 1 si el valor es mayor o igual que el umbral
    if direction=="lt":
        for obs in X:
            if obs >= value_percentil[1]:
                predict.append(1)
            else:
                predict.append(0)

    # Si la dirección es "gt" (mayor que), predecir 1 si el valor es menor o igual que el umbral
    elif direction=="gt":
         for obs in X:
            if obs <= value_percentil[1]:
                predict.append(1)
            else:
                predict.append(0)

    return predict

#################################################################################################################
# Función list_predictions: Devuelve un diccionario con las predicciones para cada par (percentil, valor)
def list_predictions(vector_x, steps = 1, direction = "gt"):
    dict_parejas = {}
    # Calcular la lista de percentiles para el conjunto de datos
    lista_percentiles = percentiles(vector_x, steps = steps)
    # Generar las predicciones para cada par (percentil, valor) y guardarlas en un diccionario
    for i in lista_percentiles:
        dict_parejas[i] = prediction(vector_x, i, direction)
    return dict_parejas

#################################################################################################################
# Función kaminsky_metricas: Calcula diversas métricas de evaluación para cada par (percentil, valor)
def kaminsky_metricas(y_real, vector_x, steps = 1, direction = "gt"):
    # Obtener el diccionario de predicciones para cada par (percentil, valor)
    dictpar_pred = list_predictions(vector_x, steps = steps, direction = direction)
    pd_final = pd.DataFrame()
    # Calcular las métricas de evaluación para cada par (percentil, valor) y guardarlas en un DataFrame
    for pareja in dictpar_pred.keys():
        metricas = scores(y_real, dictpar_pred[pareja])
        metricas.columns  = [pareja]
        pd_final = pd.concat([pd_final, metricas], axis = 1)
    return pd_final.T

#################################################################################################################
# Función kaminsky_max: Encuentra el umbral que maximiza el score SP (u otra métrica) en el conjunto de datos
def kaminsky_max(y_real, vector_x, steps = 1, direction = "gt", crit_max = "SP_score", escogencia = "last"):
    # Calcular las métricas de evaluación para cada par (percentil, valor)
    dataframe = kaminsky_metricas(y_real, vector_x, steps = steps, direction = direction)
    # Filtrar las métricas de evaluación para solo mantener la métrica especificada en crit_max
    dataframe = dataframe[[crit_max]]
    # Encontrar el valor máximo de la métrica especificada en crit_max
    max = dataframe[crit_max].max()
    # Filtrar el DataFrame para solo mantener los umbrales que alcanzan el valor máximo de crit_max
    datafmax  = dataframe[dataframe[crit_max]==max]

    tupla_max = None
    # Seleccionar el umbral máximo de acuerdo con el criterio especificado en escogencia
    if escogencia   == "last":
        tupla_max = datafmax.index[-1]
    elif escogencia == "first":
        tupla_max = datafmax.index[0]
    elif escogencia == "middle":
        tupla_max = round((datafmax.index[-1]+ datafmax.index[0])/2)

    # Extraer el percentil y el valor del umbral máximo
    percentil_max = tupla_max[0]
    valor_max = tupla_max[1]

    # Devolver el valor máximo, el score SP máximo (o cualquier otra métrica) y el percentil máximo
    return valor_max, max, percentil_max

### 3. Running model over FISLAC data

In [6]:
dfi = pd.read_excel("FISLAC_update.xlsx") #IMPORT DATAFRAME (FISLAC DATA)
dictv = pd.read_excel("Dictionary of variables - FISLAC 2022.xlsx") #IMPORT DICITIONARY (GET THE SAFETY DIR)
dictv = dictv[["id", " safety_dir"]]
dictv.columns = ["id", "safety_dir"]

In [7]:
# PAISES_V = ["COL", "URU", "CHL", "USA", "BRA"]
# VARIABLES22 = ["DEBT_NET_GDP", "DEBT_GDP", "BROAD_MONEY_GDP", "POTOUTPUT", "OUTPUTGAP", "NGS_GDP", "INFLATION", "GDP", "GDP_PPP", "EXT_DEBT_GDP"]

In [11]:
y = [1,0,0,0,0,0,0,0,0,1]
x = [2,3,4,3,3,3,2,10,2,9]

In [13]:
kaminsky_max(y, x, steps = 10, direction = dir, crit_max = criteria_max, escogencia = orden_escogencia)

(2.6999999999999997, 0.25, 30)

In [8]:
#PARAMETROS

saltos_percentiles = 5
criteria_max = "SP_score"
orden_escogencia = "last"
rezago = 1
anio_i = 2012
anio_f = 2021

df = dfi[(dfi.DATE>=anio_i) & (dfi.DATE<=anio_f)] #CONSTRAIN YEARS
ISO3 = df.ISO3.unique() #COUNTRIES
VARS = dict(zip(dictv.id, dictv.safety_dir)) #VARIABLE DICTIONARY WITH SAFETY DIRECTION
PAISES_V = df.ISO3.unique()

#LOOP

paises, variables, sdir, threshold, spvalue, percentil, median = [], [], [], [], [], [], []


for pais in PAISES_V[0:3]:
    
    dfpais = df[df.ISO3 == pais]
    Y = dfpais["CRISIS"][rezago:]


    for variable in VARS:

        dir = VARS[variable]

        try:
            x = dfpais[variable]
            if rezago>0:
                x = x[:-rezago]
            valork = kaminsky_max(Y, x, steps = saltos_percentiles, direction = dir, crit_max = criteria_max, escogencia = orden_escogencia)
        except:
            valork = "VVVV"

        paises.append(pais); variables.append(variable); sdir.append(dir); threshold.append(valork[0]); spvalue.append(valork[1]); percentil.append(valork[2])
        median.append(x.median())

df_valores = pd.DataFrame(zip(paises, variables, sdir, threshold, spvalue, percentil, median), 
    columns = ["ISO3", "id", "safety_dir", "Threshold", "Signal Power", "Percentile", "Median"])

c:\Users\War\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\War\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\War\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\War\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0

### 4. Arranging to SQL

#### 4.1 Country-level DB

In [116]:
dict_factsubfact = pd.read_excel("Dictionary of variables - FISLAC 2022.xlsx") #IMPORT DICITIONARY (GET THE SAFETY DIR)
dict_factsubfact = dict_factsubfact[[" factor", " name_en", " subfactor", "id"]]

In [117]:
dictp = dfi[["ISO3", "REGION", "EMERGING","COUNTRY_NAME", "WEO_CODE"]]
dictp = dictp.drop_duplicates(["COUNTRY_NAME"])
dictp = dictp[dictp.EMERGING == 1]
dictp["EMERGING"] = "Emerging"

In [118]:
dfVF = pd.merge(df_valores, dictp, "left", "ISO3")
dfVF = pd.merge(dfVF, dict_factsubfact, "left", "id")
dfVF = dfVF[[" factor", " subfactor", " name_en", "id", "REGION", "EMERGING", "COUNTRY_NAME", "WEO_CODE", "Median", "Threshold", "Percentile", "Signal Power"]]
dfVF.columns = ['Factor', 'Subfactor', 'Variable', 'Variable Code', 'Region', 'Subregion', 'Country', 'Country Code', 'Median', 'Threshold', 'Percentile', 'Signal Power']

listad = ['Country Code', 'Median', 'Threshold', 'Percentile', 'Signal Power', 'Type I Error', 
'Type II Error', 'Precision', 'Recall', 'Accuracy', 'F1', 'F2', 'Stress Episodes', 'Tranquil Episodes']

for c in listad:
    if c not in dfVF:
        dfVF[c] = np.nan

In [120]:
dfVF.to_excel("TABLA1.xlsx", index = False)

#### 4.2 Variable level DB

In [148]:
dfv = dfVF[dfVF["Signal Power"]!="V"]
dfv_reduced = dfv[["Factor", "Subfactor", "Variable", "Variable Code"]].drop_duplicates("Variable")

#SP PROMEDIO X VARIABLE
av_signal_power = dfv.groupby("Variable")["Signal Power"].mean().reset_index()
av_signal_power.columns = ["Variable", "signal_power"]

#THR PROMEDIO X VARIABLE
av_th_power = dfv.groupby("Variable")["Threshold"].mean().reset_index()
av_th_power.columns = ["Variable", "threshold"]

#SUMA X VARIABLE
sum_sp_variable = dfv.groupby("Variable")["Signal Power"].sum().reset_index()
sum_sp_variable.columns = ["Variable", "sum_x_variable"]

#SUMA X FACTOR
sum_sp_factor = dfv.groupby("Factor")["Signal Power"].sum().reset_index()
sum_sp_factor.columns = ["Factor", "sum_sp_factor"]

#SUMA X SUBFACTOR
sum_sp_subfactor = dfv.groupby("Subfactor")["Signal Power"].sum().reset_index()
sum_sp_subfactor.columns = ["Subfactor", "sum_sp_subfactor"]


from functools import reduce # IMPORTAR FUNCIÓN
dfs = [av_signal_power, av_th_power, sum_sp_variable] #MERGE DE DATAFRAMES ANTERIORES
dfv2 = reduce(lambda left,right: pd.merge(left,right,on=["Variable"]), dfs) #MERGE DE LAS BASES
dfv2 = pd.merge(dfv2, dfv_reduced, "left", "Variable")
dfv2 = pd.merge(dfv2, sum_sp_factor, "left", "Factor")
dfv2 = pd.merge(dfv2, sum_sp_subfactor, "left", "Subfactor")


#WEIGHTS
dfv2["weights"] = dfv2["sum_x_variable"]/dfv["Signal Power"].sum()
dfv2["weights_factor"] = dfv2["sum_x_variable"]/dfv2["sum_sp_factor"]
dfv2["weights_subfactor"] = dfv2["sum_x_variable"]/(dfv2["sum_sp_subfactor"]+0.00001)


#ORGANIZAR Y ELIMINAR VARIABLES
dfv2.columns = ['Variable', 'signal_power', 'threshold', 'sum_x_variable', 'Factor',
       'Subfactor', 'variable', 'sum_sp_factor', 'sum_sp_subfactor',
       'weights', 'weights_factor', 'weights_subfactor']

dfv2 = dfv2[['Variable', 'signal_power', 'threshold', 'Factor',
       'Subfactor', 'variable', 'sum_sp_factor', 'sum_sp_subfactor',
       'weights', 'weights_factor', 'weights_subfactor']]

listad = ['Factor',  'Subfactor',  'Variable Name',  'variable',  'transformation',  'percentile',  'tp',  'tn',  
'false_alarm',  'miss',  'crises',  'non_crises',  'snr',  't1error',  't2error',  'tme',  'signal_power',  
'accuracy',  'precision',  'recall',  'f1',  'f2',  'threshold',  'sum_sp_factor',  'sum_sp_subfactor',  
'weights',  'weights_factor',  'weights_subfactor']

for var in listad:
    if var not in dfv2:
        dfv2[var] = np.nan

dfv2 = dfv2[listad]

In [150]:
dfv2.to_excel("TABLA2.xlsx", index = False)

In [ ]:
#FUNCIÓN DE SP
def sp_score(ytest, ypred):
    M = confusion_matrix(ytest, ypred)

    DEN_E1 = (M[0,1]+M[0,0])
    DEN_E2 = (M[1,0]+M[1,1])

    E1 = 0
    E2 = 0

    if DEN_E1 == 0:
        E1 = 0
    else:
        E1 = M[0,1]/DEN_E1

    if DEN_E2 == 0:
        E2 = 0
    else:
        E2 = M[1,0]/DEN_E2    
        
    SP = 1-(E1+E2)
    return SP


#FUNCIÓN DE SCORES
def scores(ytest,ypred):
    listas  = [precision_score(ytest,ypred), 
                accuracy_score(ytest,ypred), 
                recall_score(ytest,ypred),
                f1_score(ytest, ypred),
                sp_score(ytest, ypred)]
                
    A = pd.DataFrame(listas, index = ["precision_score", "accuracy_score", "recall_score", "F1_score", "SP_score"])
    return A


#(PERCENTILE, VALUE) PAIRS EXTRACTION
def percentiles(X, steps = 1):
    l = []
    for percentil in range(0,101, steps):
        value_percentil = np.percentile(X, percentil)
        pareja = (percentil, value_percentil)
        l.append(pareja)
    return l


#PERCENTILE KAMINSKY MODEL PREDICTION GIVEN (PERCENTILE, VALUE) PAIRS
def prediction(X, value_percentil, direction):

    predict = []

    if direction=="lt":                 #IF IS GREATHER THAN
        for obs in X:
            if obs>=value_percentil[1]:
                predict.append(1)
            else:
                predict.append(0)

    elif direction=="gt":               #IF IS LOWER THAN
         for obs in X:
            if obs<=value_percentil[1]:
                predict.append(1)       
            else:
                predict.append(0)

    return predict


#PREDICTION BY (PERCENTILE, VALUE) PAIRS
def list_predictions(vector_x, steps = 1, direction = "gt"):
    dict_parejas = {}
    lista_percentiles = percentiles(vector_x, steps = steps)
    for i in lista_percentiles:
        dict_parejas[i] = prediction(vector_x, i, direction)
    return dict_parejas


#METRICAS BY (PERCENTILE, VALUE) PAIRS PREDICTION
def kaminsky_metricas(y_real, vector_x, steps = 1, direction = "gt"):
    dictpar_pred = list_predictions(vector_x, steps = steps, direction = direction)
    pd_final = pd.DataFrame()
    for pareja in dictpar_pred.keys():
        metricas = scores(y_real, dictpar_pred[pareja])
        metricas.columns  = [pareja]
        pd_final = pd.concat([pd_final, metricas], axis = 1)
    return pd_final.T

#MAXIMIZATION ACORDING TO CRITERIA
def kaminsky_max(y_real, vector_x, steps = 1, direction = "gt", crit_max = "SP_score", escogencia = "last"):
    dataframe = kaminsky_metricas(y_real, vector_x, steps = steps, direction = direction)
    dataframe = dataframe[[crit_max]]
    max = dataframe[crit_max].max()
    datafmax  = dataframe[dataframe[crit_max]==max]

    tupla_max = None
    if escogencia   == "last":
        tupla_max = datafmax.index[-1]   
    elif escogencia == "first":
        tupla_max = datafmax.index[0]
    elif escogencia == "middle":
        tupla_max = round((datafmax.index[-1]+ datafmax.index[0])/2)

    percentil_max = tupla_max[0]
    valor_max = tupla_max[1]

    return valor_max, max, percentil_max  #RETORNA EL VALOR MÁXIMO, Y EL SP (SE PUEDE AGREGAR EL PARCENTIL)


# AGREGAR MEDIANA
        